In [ ]:
# | default_exp classes.DomoInstanceConfig

In [ ]:
# | export

from domolibrary.routes.instance_config import UpdateSSO_Error

In [ ]:
# | exporti
import httpx
import datetime as dt
import asyncio
from fastcore.basics import patch_to
import sys
import pandas as pd


from dataclasses import dataclass, field, asdict
from typing import List

import domolibrary.utils.DictDot as util_dd
import domolibrary.utils.chunk_execution as ce

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.client.Logger as lg

import domolibrary.routes.instance_config as instance_config_routes
import domolibrary.routes.bootstrap as bootstrap_routes
import domolibrary.routes.publish as publish_routes
import domolibrary.routes.application as application_routes

In [ ]:
# | export
@dataclass
class DomoInstanceConfig:
    """utility class that absorbs many of the domo instance configuration methods"""

    auth: dmda.DomoAuth
    allowlist: list[str] = field(default_factory=list)
    is_user_invite_notification_enabled: bool = field(default=None)
    is_invite_social_users_enabled: bool = field(default=None)

    sso_config: dict = field(default=None)

# Toggle User Invite Notifcation Enabled


In [ ]:
# |exporti


@patch_to(DomoInstanceConfig)
async def get_is_user_invite_notification_enabled(
    self: DomoInstanceConfig,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    """
    Admin > Company Settings > Admin Notifications
    Toggles whether user recieves 'You've been Domo'ed email
    """

    res = await instance_config_routes.get_is_user_invite_notifications_enabled(
        auth=auth or self.auth,
        session=session,
        debug_api=debug_api,
    )

    self.is_user_invite_notification_enabled = bool(res.response["value"])

    if return_raw:
        return res

    return self.is_user_invite_notification_enabled

#### sample implementation of is_user_invite_enabled


In [ ]:
import os
import requests


full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_password=os.environ["DOJO_PASSWORD"],
    domo_username=os.environ["DOMO_USERNAME"],
)


domo_config = DomoInstanceConfig(auth=full_auth)

await domo_config.get_is_user_invite_notification_enabled(
    auth=full_auth, debug_api=False, return_raw=False
)

True

In [ ]:
# |exporti
@patch_to(DomoInstanceConfig)
async def toggle_is_user_invite_enabled(
    self: DomoInstanceConfig,
    auth: dmda.DomoFullAuth,
    is_enabled: bool,
    debug_api: bool = False,
    debug_prn: bool = True,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    is_user_invite_notification_enabled = (
        await self.get_is_user_invite_notification_enabled(auth=auth)
    )

    if is_enabled == is_user_invite_notification_enabled:
        if debug_prn:
            print(
                f"User invite notification is already {'enabled' if is_enabled else 'disabled'} in {auth.domo_instance}"
            )
        return True

    if debug_prn:
        print(
            f"{'enabling' if is_enabled else 'disabling'} User invite notification {auth.domo_instance}"
        )

    res = await instance_config_routes.toggle_is_user_invite_enabled(
        auth=auth or self.auth,
        is_enabled=is_enabled,
        session=session,
        debug_api=debug_api,
    )

    if return_raw:
        return res

    return await self.get_is_user_invite_notification_enabled(auth=auth)

### Sample implementation of toogle User invite enabled


In [ ]:
import os

auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_username=os.environ["DOMO_USERNAME"],
    domo_password=os.environ["DOJO_PASSWORD"],
)

domo_config = DomoInstanceConfig(auth=auth)

await domo_config.toggle_is_user_invite_enabled(auth=auth, is_enabled=True)

User invite notification is already enabled in domo-community


True

# Toggle Social Users


In [ ]:
# |exporti


@patch_to(DomoInstanceConfig)
async def get_is_invite_social_users_enabled(
    self: DomoInstanceConfig,
    auth: dmda.DomoFullAuth,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    import domolibrary.classes.DomoBootstrap as dmbp

    bs = dmbp.DomoBootstrap(auth=auth)
    customer_id = await bs.get_customer_id()

    res = await instance_config_routes.get_is_invite_social_users_enabled(
        auth=auth or self.auth,
        customer_id=customer_id,
        session=session,
        debug_api=debug_api,
    )

    self.is_invite_social_users_enabled = bool(res.response["enabled"])

    if return_raw:
        return res

    return res.response["enabled"]

#### sample implementation of is_invite_social_users_enabled


In [ ]:
import os
import requests


full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_password=os.environ["DOJO_PASSWORD"],
    domo_username=os.environ["DOMO_USERNAME"],
)


domo_config = DomoInstanceConfig(auth=full_auth)

await domo_config.get_is_invite_social_users_enabled(
    auth=full_auth, debug_api=False, return_raw=False
)

True

In [ ]:
# |exporti
@patch_to(DomoInstanceConfig)
async def toggle_social_users(
    self: DomoInstanceConfig,
    auth: dmda.DomoFullAuth,
    is_enabled: bool,
    debug_api: bool = False,
    debug_prn: bool = True,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    is_invite_social_users_enabled = await self.get_is_invite_social_users_enabled(
        auth=auth
    )

    if is_enabled == is_invite_social_users_enabled:
        if debug_prn:
            print(
                f"invite social users is already {'enabled' if is_enabled else 'disabled'} in {auth.domo_instance}"
            )
        return True

    if debug_prn:
        print(
            f"{'enabling' if is_enabled else 'disabling'} invite social users {auth.domo_instance}"
        )

    res = await instance_config_routes.toggle_social_users(
        auth=auth or self.auth,
        is_enabled=is_enabled,
        session=session,
        debug_api=debug_api,
    )

    if return_raw:
        return res

    return await self.get_is_invite_social_users_enabled()

#### sample implementation of toggle_social_users


In [ ]:
import os


auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_username=os.environ["DOMO_USERNAME"],
    domo_password=os.environ["DOJO_PASSWORD"],
)

domo_config = DomoInstanceConfig(auth=auth)

await domo_config.toggle_social_users(auth=auth, is_enabled=True)

invite social users is already enabled in domo-community


True

# SSO Configuration


In [ ]:
# | exporti
py310 = sys.version_info.minor >= 10 or sys.version_info.major > 3

In [ ]:
# | export

# class SSOConfig_InstantiationError(de.DomoError):
#     def __init__(self, domo_instance, parent_class, function_name, message="invalid data types, check attribute types"):

#         super().__init__(
#             domo_instance=domo_instance,
#             message=message,
#             parent_class=parent_class,
#             function_name=function_name)


@dataclass(**({"slots": True} if py310 else {}))
class SSO_Config:
    auth: dmda.DomoAuth = field(repr=False)

    login_enabled: bool = None  # False
    idp_enabled: bool = None  # False
    import_groups: bool = None  # False
    require_invitation: bool = None  # False
    enforce_allowlist: bool = None  # False
    skip_to_idp: bool = None  # False
    auth_request_endpoint: str = None
    token_endpoint: str = None
    user_info_endpoint: str = None
    public_key: str = None
    redirect_url: str = None
    certificate: str = None
    override_sso: bool = None  # False
    override_embed: bool = None  # False
    # "https://{domo_instance}}.domo.com/auth/oidc"
    well_known_config: str = None
    assertion_endpoint: str = None
    ingest_attributes: bool = None  # False

    # def __post_init__(self):
    #     self.override_sso = self.override_sso or f"https://{auth.domo_instance}.domo.com/auth/oidc"

    @classmethod
    def _from_json(cls, auth: dmda.DomoAuth, obj: dict):
        dd = obj

        if not isinstance(obj, util_dd.DictDot):
            dd = util_dd.DictDot(obj)

        return cls(
            auth=auth,
            login_enabled=dd.loginEnabled,
            idp_enabled=dd.idpEnabled,
            import_groups=dd.importGroups,
            require_invitation=dd.requireInvitation,
            enforce_allowlist=dd.enforceWhitelist,
            skip_to_idp=dd.skipToIdp,
            auth_request_endpoint=dd.authRequestEndpoint,
            token_endpoint=dd.tokenEndpoint,
            user_info_endpoint=dd.userInfoEndpoint,
            public_key=dd.publicKey,
            redirect_url=dd.redirectUrl,
            certificate=dd.certificate,
            override_sso=dd.overrideSSO,
            override_embed=dd.overrideEmbed,
            well_known_config=dd.wellKnownConfig,
            assertion_endpoint=dd.assertionEndpoint,
            ingest_attributes=dd.ingestAttributes,
        )

    def add_attribute(self, overwrite_existing: bool = False, **kwargs):
        [
            setattr(self, key, value)
            for key, value in kwargs.items()
            if value is not None
        ]
        return self

        # except TypeError as e:
        #     traceback_details = lg.get_traceback(num_stacks_to_drop=1)

        #     raise SSOConfig_InstantiationError(
        #         domo_instance=self.auth.domo_instance,
        #         parent_class=self.__class__.__name__,
        #         function_name=traceback_details.function_name)

    def to_json(self, is_include_undefined: bool = False):
        r = {
            "loginEnabled": self.login_enabled,
            "idpEnabled": self.idp_enabled,
            "importGroups": self.import_groups,
            "requireInvitation": self.require_invitation,
            "enforceWhitelist": self.enforce_allowlist,
            "skipToIdp": self.skip_to_idp,
            "authRequestEndpoint": self.auth_request_endpoint,
            "tokenEndpoint": self.token_endpoint,
            "userInfoEndpoint": self.user_info_endpoint,
            "publicKey": self.public_key,
            "redirectUrl": self.redirect_url,
            "certificate": self.certificate,
            "overrideSSO": self.override_sso,
            "overrideEmbed": self.override_embed,
            "wellKnownConfig": self.well_known_config,
            "assertionEndpoint": self.assertion_endpoint,
            "ingestAttributes": self.ingest_attributes,
        }

        if not is_include_undefined:
            return {key: value for key, value in r.items() if value is not None}

        return r

In [ ]:
from dataclasses import asdict

sso_config = SSO_Config(enforce_allowlist=True, login_enabled=False, auth=auth)

sso_config.add_attribute(skip_to_idp=True, certificate="abc")

sso_config.to_json()

{'loginEnabled': False,
 'enforceWhitelist': True,
 'skipToIdp': True,
 'certificate': 'abc'}

In [ ]:
# | exporti
@patch_to(DomoInstanceConfig)
async def get_sso_config(
    self: DomoInstanceConfig,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    return_raw: bool = False,
):
    res = await instance_config_routes.get_sso_config(
        auth=self.auth,
        session=session,
        parent_class=self.__class__.__name__,
        debug_api=debug_api,
        debug_num_stacks_to_drop=2,
    )

    if return_raw:
        return res

    self.sso_config = SSO_Config._from_json(auth=auth, obj=res.response)

    return self.sso_config

#### sample implementation of get_sso_configuration


In [ ]:
import os
import requests

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_config = DomoInstanceConfig(auth=token_auth)

await domo_config.get_sso_config()

domo_config.sso_config.to_json()

{'loginEnabled': False,
 'idpEnabled': False,
 'importGroups': False,
 'requireInvitation': False,
 'enforceWhitelist': False,
 'skipToIdp': False,
 'authRequestEndpoint': '',
 'tokenEndpoint': '',
 'userInfoEndpoint': '',
 'publicKey': '',
 'redirectUrl': '',
 'certificate': '',
 'overrideSSO': False,
 'overrideEmbed': False,
 'wellKnownConfig': ''}

## Update SSO Config


In [ ]:
# | exporti
@patch_to(DomoInstanceConfig)
async def update_sso_config(
    self: DomoInstanceConfig,
    login_enabled: bool = None,  # False
    idp_enabled: bool = None,  # False
    import_groups: bool = None,  # False
    require_invitation: bool = None,  # False
    enforce_allowlist: bool = None,  # False
    skip_to_idp: bool = None,  # False
    auth_request_endpoint: str = None,
    token_endpoint: str = None,
    user_info_endpoint: str = None,
    public_key: str = None,
    redirect_url: str = None,
    certificate: str = None,
    override_sso: bool = None,  # False
    override_embed: bool = None,  # False
    # "https://{domo_instance}}.domo.com/auth/oidc"
    well_known_config: str = None,
    assertion_endpoint: str = None,
    ingest_attributes: bool = None,  # False
    debug_is_test: bool = False,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
):
    update_config = await self.get_sso_config()

    update_config.add_attribute(
        overwrite_existing=True,
        login_enabled=login_enabled,
        idp_enabled=idp_enabled,
        import_groups=import_groups,
        require_invitation=require_invitation,
        enforce_allowlist=enforce_allowlist,
        skip_to_idp=skip_to_idp,
        auth_request_endpoint=auth_request_endpoint,
        token_endpoint=token_endpoint,
        user_info_endpoint=user_info_endpoint,
        public_key=public_key,
        redirect_url=redirect_url,
        certificate=certificate,
        override_sso=override_sso,
        override_embed=override_embed,
        well_known_config=well_known_config,
        assertion_endpoint=assertion_endpoint,
        ingest_attributes=ingest_attributes,
    )

    config_body = update_config.to_json()

    if debug_is_test:
        print("⚗️⚠️ This is a test, SSO Config will not be updated")
        return config_body

    res = await instance_config_routes.update_sso_config(
        auth=self.auth,
        config_body=config_body,
        parent_class=self.__class__.__name__,
        session=session,
        debug_api=debug_api,
        debug_num_stacks_to_drop=2,
    )

    # await asyncio.sleep(3)

    await self.get_sso_config()

    errors_obj = {
        update_key: f"expected_value: {str(update_value)  } , current_value: { str(self.sso_config[update_key])}"
        for update_key, update_value in asdict(update_config).items()
        if asdict(self.sso_config)[update_key] != update_value
    }

    if len(errors_obj.keys()) > 0:
        raise instance_config_routes.UpdateSSO_Error(
            domo_instance=self.auth.domo_instance,
            config_body=errors_obj,
            function_name=res.traceback_details.function_name,
            parent_class=self.__class__.__name,
        )

    return self.sso_config

# sample implementation of update_sso_config


In [ ]:
import os
import requests

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_config = DomoInstanceConfig(auth=token_auth)
try:
    await domo_config.update_sso_config(
        require_invitation=True,
        override_embed="abc",
        # debug_is_test=True,
        debug_api=False,
    )
except UpdateSSO_Error as e:
    print(e)

🛑  UpdateSSO_Error 🛑 - function: DomoInstanceConfig.update_sso_config || status 400 || failed to set config to loginEnabled : False || idpEnabled : False || importGroups : False || requireInvitation : True || enforceWhitelist : False || skipToIdp : False || authRequestEndpoint :  || tokenEndpoint :  || userInfoEndpoint :  || publicKey :  || redirectUrl :  || certificate :  || overrideSSO : False || overrideEmbed : abc || wellKnownConfig :  at domo-community


## Get Publications


In [ ]:
# | exporti
@patch_to(DomoInstanceConfig, cls_method=True)
async def get_publications(
    cls: DomoInstanceConfig,
    auth: dmda.DomoFullAuth,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    import domolibrary.classes.DomoPublish as dmpb

    res = await publish_routes.search_publications(
        auth=auth, debug_api=debug_api, session=session
    )
    if debug_api:
        print("Getting Publish jobs")

    if res.status == 200 and not return_raw:
        return await ce.gather_with_concurrency(
            n=60,
            *[
                dmpb.DomoPublication.get_from_id(
                    publication_id=job.get("id"), auth=auth
                )
                for job in res.response
            ],
        )

    if res.status == 200 and return_raw:
        return res.response

### sample implementation of get_publications


In [ ]:
import os


auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_username=os.environ["DOMO_USERNAME"],
    domo_password=os.environ["DOJO_PASSWORD"],
)

list_pub = await DomoInstanceConfig.get_publications(auth=auth)
print(list_pub)

[DomoPublication(id='095426a1-2c35-468b-bb96-36c516382f19', name='Ellibot', description='', is_v2=True, created_dt=datetime.datetime(2023, 11, 10, 18, 5, 10, 961000), subscription_authorizations=[DomoPublication_Subscription(subscription_id='173ee64d-4f68-4d40-b634-22b5332f4e45', publication_id='095426a1-2c35-468b-bb96-36c516382f19', domain='domo-community-ellibot.domo.com', created_dt=datetime.datetime(2023, 11, 10, 18, 5, 10, 961000))], content=[DomoPublication_Content(content_id='c61531b6-81ac-45ec-9a77-d41194406097', entity_type='DATASET', entity_id='f6210193-5322-4481-a1de-d07aed544050', entity_domain='domo-community.domo.com', is_v2=True, is_direct_content=True)], content_page_id_ls=[], content_dataset_id_ls=['f6210193-5322-4481-a1de-d07aed544050'], lineage=DomoLineage(page_id_ls=[], card_id_ls=[], dataflow_id_ls=[], dataset_id_ls=[], entity_ls=[])), DomoPublication(id='99a4f772-f0e0-47a6-873a-5870d03b9134', name='Landing Page QA', description='', is_v2=True, created_dt=datetime.

# Allowlist


In [ ]:
# | exporti
@patch_to(DomoInstanceConfig)
async def get_allowlist(
    self: DomoInstanceConfig,
    auth: dmda.DomoFullAuth = None,  # get_allowlist requires full authentication
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
    debug_api: bool = False,
) -> list[str]:
    """retrieves the allowlist for an instance"""

    auth = auth or self.auth

    res = None
    loop = 0

    while not res and loop <= 5:
        try:
            res = await instance_config_routes.get_allowlist(
                auth=auth, debug_api=debug_api, session=session
            )
        except Exception as e:
            print(e)
        finally:
            loop += 1

    if return_raw:
        return res

    if not res.is_success:
        return None

    allowlist = res.response.get("addresses")

    self.allowlist = allowlist

    return allowlist

#### sample implementation of get_allowlist


In [ ]:
import os

# this route requires full auth
# auth = dmda.DomoTokenAuth(domo_instance = 'domo-community', domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])

auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_username=os.environ["DOMO_USERNAME"],
    domo_password=os.environ["DOJO_PASSWORD"],
)

dmicnfg = DomoInstanceConfig(auth=auth)

await dmicnfg.get_allowlist(auth=auth)

['0.0.0.0/0', '192.168.0.1', '20.171.48.165']

## ALTER the allowlist

- The updating the allowlist will automatically include the IP Address that submits the set_allowlist request. This prevents users from accidentally locking themselves out of domo.
- the set_allowlist API always does a SET command, by default there is no UPDATE or INSERT option. This class-based implementation uses a combination of get_allowlist, MERGE and set_allowlist to perform an UPSERT equivalent.


In [ ]:
# |exporti
@patch_to(DomoInstanceConfig)
async def set_allowlist(
    self: DomoInstanceConfig,
    ip_address_ls: list[str],
    debug_api: bool = False,
    auth: dmda.DomoFullAuth = None,
    session: httpx.AsyncClient = None,
):
    auth = auth or self.auth

    await instance_config_routes.set_allowlist(
        auth=auth, ip_address_ls=ip_address_ls, debug_api=debug_api, session=session
    )

    return await self.get_allowlist(auth=auth, debug_api=debug_api, session=session)


@patch_to(
    DomoInstanceConfig,
)
async def upsert_allowlist(
    self: DomoInstanceConfig,
    ip_address_ls: list[str],
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    auth: dmda.DomoAuth = None,
):
    exist_ip_address_ls = await self.get_allowlist(
        auth=auth, debug_api=debug_api, session=session
    )
    ip_address_ls += exist_ip_address_ls

    return await self.set_allowlist(
        auth=auth,
        ip_address_ls=list(set(ip_address_ls)),
        debug_api=debug_api,
        session=session,
    )

#### sample implementation of set and upsert allowlist


In [ ]:
import os

# this route requires full auth
# auth = dmda.DomoTokenAuth(domo_instance = 'domo-community', domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])

auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_username=os.environ["DOMO_USERNAME"],
    domo_password=os.environ["DOJO_PASSWORD"],
)

dmicnfg = DomoInstanceConfig(auth=auth)
try:
    await dmicnfg.set_allowlist(auth=auth, ip_address_ls=["0.0.0.0/0"])
except Exception as e:
    print(e)

In [ ]:
import os

# this route requires full auth
# auth = dmda.DomoTokenAuth(domo_instance = 'domo-community', domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])

auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_username=os.environ["DOMO_USERNAME"],
    domo_password=os.environ["DOJO_PASSWORD"],
)

dmicnfg = DomoInstanceConfig(auth=auth)
try:
    await dmicnfg.upsert_allowlist(auth=auth, ip_address_ls=["192.168.0.1"])
except Exception as e:
    print(e)

## Domo Grants


In [ ]:
# | exporti
@patch_to(DomoInstanceConfig)
async def get_grants(
    self: DomoInstanceConfig,
    auth: dmda.DomoAuth = None,
    debug_prn: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    import domolibrary.classes.DomoGrant as dmg

    auth = auth or self.auth

    return await dmg.DomoGrants.get_grants(
        auth=auth, return_raw=return_raw, session=session, debug_api=debug_api
    )

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-community",
)

dmicnfg = DomoInstanceConfig(auth=token_auth)

res = await dmicnfg.get_grants(debug_prn=True)

pd.DataFrame(res[0:5])

,id,display_group,title,depends_on_ls,description,role_membership_ls
0,authorization.roles,COMPANY,Manage all roles,[ui.admin],"Create, edit and delete custom roles. Assign a...","[1, 275763436, 810756122]"
1,entity.attribute.admin,COMPANY,Manage Certified Attributes,[ui.admin],"Create, edit and delete certified attributes","[1, 810756122]"
2,audit,COMPANY,View activity logs,[ui.admin],View and export audit logs.,"[1, 275763436, 810756122, 1927158482, 15631017..."
3,customer.edit,COMPANY,Manage all company settings,[ui.admin],"Manage company metadata, authentication rules,...","[1, 275763436, 810756122]"
4,developer.token.manage,COMPANY,Manage all access tokens and API clients,[ui.admin],Create and delete any access tokens and API cl...,"[1, 275763436, 810756122]"


## Domo Roles


In [ ]:
# | exporti
@patch_to(DomoInstanceConfig)
async def get_roles(
    self,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    return_raw: bool = False,
    session: httpx.AsyncClient = None,
):
    import domolibrary.classes.DomoRole as dmr

    auth = auth or self.auth

    return await dmr.DomoRoles.get_roles(
        auth=auth, debug_api=debug_api, return_raw=return_raw, session=session
    )

#### sample implementation of get_roles


In [ ]:
import domolibrary.client.DomoAuth as dmda
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

dmdic = DomoInstanceConfig(auth=token_auth)

res = await dmdic.get_roles(return_raw=False)

pd.DataFrame(res)

,auth,id,name,description,is_system_role,is_default_role,grant_ls,membership_ls
0,"{'domo_instance': 'domo-community', 'domo_acce...",1,Admin,Full access to everything,True,False,[],[]
1,"{'domo_instance': 'domo-community', 'domo_acce...",2,Privileged,Full access except for editing users and setti...,True,True,[],[]
2,"{'domo_instance': 'domo-community', 'domo_acce...",3,Editor,"Can edit Cards, Pages, DataSets, and Dataflows",True,False,[],[]
3,"{'domo_instance': 'domo-community', 'domo_acce...",4,Participant,Read only access to Cards and Pages,True,False,[],[]
4,"{'domo_instance': 'domo-community', 'domo_acce...",5,Social,Access limited to social features,True,False,[],[]
5,"{'domo_instance': 'domo-community', 'domo_acce...",104295428,super_admin_v3,upsert via DomoLibrary,0,False,[],[]
6,"{'domo_instance': 'domo-community', 'domo_acce...",275763436,manual_super_admin,Full access to everything - h9,0,False,[],[]
7,"{'domo_instance': 'domo-community', 'domo_acce...",810756122,super_admin,last updated - 2023-11-10,0,False,[],[]
8,"{'domo_instance': 'domo-community', 'domo_acce...",1563101750,dl_department_admin,deployed via domo_library script - updated 202...,0,False,[],[]
9,"{'domo_instance': 'domo-community', 'domo_acce...",1662852841,dl_test,deployed via domo_library script - updated 202...,0,False,[],[]


## Authorized Domains


In [ ]:
# | exporti


@patch_to(DomoInstanceConfig)
async def get_authorized_domains(
    self: DomoInstanceConfig,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    auth = auth or self.auth

    res = await instance_config_routes.get_authorized_domains(
        auth=auth, debug_api=debug_api, session=session, return_raw=return_raw
    )

    if return_raw:
        return res

    return res.response

#### sample implementation of get_authorized_domains


In [ ]:
import domolibrary.client.DomoAuth as dmda
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

dmdic = DomoInstanceConfig(auth=token_auth)

await dmdic.get_authorized_domains(return_raw=False)

[]

In [ ]:
# | exporti
@patch_to(DomoInstanceConfig, cls_method=True)
async def set_authorized_domains(
    cls: DomoInstanceConfig,
    auth: dmda.DomoAuth,
    authorized_domains: list[str],
    debug_prn: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    if debug_prn:
        print(f'🌡️ setting authorized domain with {",".join(authorized_domains)}')

    res = await instance_config_routes.set_authorized_domains(
        auth=auth,
        authorized_domain_ls=authorized_domains,
        debug_api=debug_api,
        session=session,
    )

    if res.status == 200 or res.status == 204:
        dmdic = DomoInstanceConfig(auth=auth)
        res.response = {
            "authorized_domains": await dmdic.get_authorized_domains(
                debug_api=debug_api
            ),
            "status": 200,
        }

    return res


@patch_to(DomoInstanceConfig, cls_method=True)
async def upsert_authorized_domains(
    cls: DomoInstanceConfig,
    auth: dmda.DomoAuth,
    authorized_domains: list[str],
    debug_prn: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    existing_domains = await cls.get_authorized_domains(auth=auth, debug_api=debug_api)

    authorized_domains += existing_domains

    if debug_prn:
        print(f'🌡️ upsertting authorized domain to {",".join(authorized_domains)}')

    return await cls.set_authorized_domains(
        auth=auth,
        authorized_domains=authorized_domains,
        debug_api=debug_api,
        session=session,
    )

In [ ]:
# | exporti
@patch_to(DomoInstanceConfig, cls_method=True)
async def get_applications(
    cls,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
    debug_num_stacks_to_drop=2,
):
    import domolibrary.classes.DomoApplication as dmapp

    res = await application_routes.get_applications(
        auth=auth,
        debug_api=debug_api,
        session=session,
        parent_class=cls.__name__,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if return_raw:
        return res

    if res.status != 200:
        return res

    return [dmapp.DomoApplication._from_json(job) for job in res.response]

#### sample implementation of get_applications()

In [ ]:
import domolibrary.client.DomoAuth as dmda
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

dmdic = DomoInstanceConfig(auth=auth)

await dmdic.get_applications(auth=auth, return_raw=False)

[DomoApplication(id='25a97e0c-df6b-11eb-ba80-0242ac130004', customer_id='mmmm-0012-0200', name='Toolkit: PDP Automation', description='PDP Automation', version='1.0.81_1', execution_class='com.domo.executor.pdp.PDPAutomationApplication', grants=['pipeline.executor.job.create', 'dataset.manage'], jobs=None),
 DomoApplication(id='4ddbf5d7-6441-4eb3-b5aa-97707cae2d2b', customer_id='mmmm-0012-0200', name='Toolkit: Schema Management', description='Schema Management', version='1.0.25_1', execution_class='com.domo.executor.schema.SchemaManagement', grants=['dataset.admin'], jobs=None),
 DomoApplication(id='e37a6942-9c0f-485a-8288-4fe95e10b23d', customer_id='mmmm-0012-0200', name='Toolkit: DataSet S3 Backup', description='DataSet S3 Backup', version='1.0.7_1', execution_class='com.domo.executor.backup.DataSetBackupExecutor', grants=['dataset.admin', 'search.allentities.query'], jobs=None),
 DomoApplication(id='b52f3c80-2642-4dcb-b874-b327326021b0', customer_id='mmmm-0012-0200', name='Toolkit: 

In [ ]:
# | exporti
@patch_to(DomoInstanceConfig)
async def generate_applications_report(
    self,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
    debug_num_stacks_to_drop=2,
):
    import domolibrary.classes.DomoApplication as dmapp

    domo_apps = await self.get_applications(auth=self.auth, debug_api=debug_api)

    df = pd.DataFrame([app.__dict__ for app in domo_apps])
    df["domo_instance"] = self.auth.domo_instance

    df.drop(columns=["auth"], inplace=True)
    df.rename(
        columns={
            "id": "application_id",
            "name": "application_name",
            "description": "application_description",
            "version": "application_version",
        },
        inplace=True,
    )

    return df.sort_index(axis=1)

#### sample implementation of generate_appications_report

In [ ]:
import domolibrary.client.DomoAuth as dmda
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_config = DomoInstanceConfig(auth=auth)

await domo_config.generate_applications_report()

,application_description,application_id,application_name,application_version,customer_id,domo_instance,execution_class,grants,jobs,jobs_schedule
0,PDP Automation,25a97e0c-df6b-11eb-ba80-0242ac130004,Toolkit: PDP Automation,1.0.81_1,mmmm-0012-0200,domo-community,com.domo.executor.pdp.PDPAutomationApplication,"[pipeline.executor.job.create, dataset.manage]",None,None
1,Schema Management,4ddbf5d7-6441-4eb3-b5aa-97707cae2d2b,Toolkit: Schema Management,1.0.25_1,mmmm-0012-0200,domo-community,com.domo.executor.schema.SchemaManagement,[dataset.admin],None,None
2,DataSet S3 Backup,e37a6942-9c0f-485a-8288-4fe95e10b23d,Toolkit: DataSet S3 Backup,1.0.7_1,mmmm-0012-0200,domo-community,com.domo.executor.backup.DataSetBackupExecutor,"[dataset.admin, search.allentities.query]",None,None
3,Toolkit: User Automation,b52f3c80-2642-4dcb-b874-b327326021b0,Toolkit: User Automation,1.51_2,mmmm-0012-0200,domo-community,com.domo.executor.userautomation.UserAutomatio...,"[pipeline.executor.job.create, dataset.manage,...",None,None
4,Group Management,2f6573a5-97d8-4e27-b0fd-3c0f2313a3c8,Toolkit: Group Management,1.0.42_1,mmmm-0012-0200,domo-community,com.domo.executor.automation.GroupAutomationEx...,"[pipeline.executor.job.create, dataset.admin, ...",None,None
5,Triggers Scheduled reports based on triggered ...,bf651f89-2b76-4290-a5d8-d4dbcddc86a3,Toolkit: Triggered Reports,1.0.7_1,mmmm-0012-0200,domo-community,com.domo.executor.reports.ReportTriggerExecutor,"[pipeline.executor.job.create, dataset.manage,...",None,None
6,Remote Domo Stats,50e7230f-d2f2-42e2-a208-d94c8ae9f64c,Toolkit: Remote Domo Stats,1.2.389_1,mmmm-0012-0200,domo-community,com.domo.executor.subscriberstats.SubscriberSt...,"[developer.token.manage, pipeline.executor.job...",None,None
7,DataSet Tag Automation,a99c3fd8-a0f6-4d06-9a1d-74f3d12293d4,Toolkit: DataSet Tag Automation,1.0.32_1,mmmm-0012-0200,domo-community,com.domo.executor.tags.ResourceTagApplication,"[dataflow.admin, dataset.admin, search.allenti...",None,None
8,Geocoder Executor Service,068a1c2d-7a1a-449d-82d9-5ab54e2eb16e,Geocoder Executor,1.0.75_1,mmmm-0012-0200,domo-community,com.domo.executor.geocoder.GeocoderApplication,[pipeline.executor.job.create],None,None
9,Virtual Dataset Sharing,69a16dc5-1d0e-45a6-8e15-459d8c0b1b42,Toolkit: Virtual Dataset Sharing,1.0.21_1,mmmm-0012-0200,domo-community,com.domo.executor.dataset.DataSetSharingApplic...,"[pipeline.executor.job.create, dataset.manage]",None,None


In [ ]:
# #| export
# @dataclass
# class DomoConnector:
#     id: str
#     label: str
#     title: str
#     sub_title: str
#     description: str
#     create_date: dt.datetime
#     last_modified: dt.datetime
#     publisher_name: str
#     writeback_enabled: bool
#     tags: list[str] = field(default_factory=list)
#     capabilities: list[str] = field(default_factory=list)

#     @classmethod
#     def _from_str(cls, obj):
#         dd = DictDot(obj)

#         return cls(
#             id=dd.databaseId,
#             label=dd.label,
#             title=dd.title,
#             sub_title=dd.subTitle,
#             description=dd.description,
#             create_date=cd.convert_epoch_millisecond_to_datetime(
#                 dd.createDate),
#             last_modified=cd.convert_epoch_millisecond_to_datetime(
#                 dd.lastModified),
#             publisher_name=dd.publisherName,
#             writeback_enabled=dd.writebackEnabled,
#             tags=dd.tags,
#             capabilities=dd.capabilities
#         )

In [ ]:
# @classmethod
# async def get_connectors(cls, auth: DomoAuth,
#                          session: httpx.AsyncClient = None,
#                          debug: bool = False,
#                          limit=100,
#                          ):

#     is_close_session = False
#     if not session:
#         is_close_session = True
#         session = httpx.AsyncClient()

#     def arr_fn(res):
#         # pprint(res.response)
#         return res.response.get('searchObjects')

#     def alter_maximum_fn(res):
#         return res.response.get('totalResultCount')

#     body = {
#         "count": limit,
#         "offset": 0,
#         "hideSearchObjects": True,
#         "combineResults": False,
#         "entities": ["CONNECTOR"],
#         "query": "*"}

#     obj_list = await datacenter_routes.search_datacenter(
#         auth=auth,
#         arr_fn=arr_fn,
#         alter_maximum_fn=alter_maximum_fn,
#         body=body,
#         session=session,
#         limit=limit,
#         debug=debug)

#     if is_close_session:
#         await session.aclose()

#     return [DomoConnector._from_str(obj) for obj in obj_list]

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()

!nbqa black 50_DomoInstanceConfig.ipynb

All done! ✨ 🍰 ✨
1 file left unchanged.
